In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/Test Data.csv
/kaggle/input/dataset/Training Data.csv
/kaggle/input/dataset/Sample Prediction Dataset.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import warnings

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, log_loss
from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier

In [3]:
train_data = pd.read_csv("../input/dataset/Training Data.csv")
test_data = pd.read_csv("../input/dataset/Test Data.csv")

In [4]:
train_data.shape

(252000, 13)

In [5]:
train_data.isnull().sum()

Id                     0
income                 0
age                    0
experience             0
married                0
house_ownership        0
car_ownership          0
profession             0
city                   0
state                  0
current_job_years      0
current_house_years    0
risk_flag              0
dtype: int64

In [6]:
test_data.isnull().sum()

id                     0
income                 0
age                    0
experience             0
married                0
house_ownership        0
car_ownership          0
profession             0
city                   0
state                  0
current_job_years      0
current_house_years    0
dtype: int64

In [7]:
train_data.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [8]:
train_data["profession"]=pd.factorize(train_data.profession)[0]
train_data["city"]=pd.factorize(train_data.city)[0]
train_data["state"]=pd.factorize(train_data.state)[0]
train_data["married"]=pd.factorize(train_data.married)[0]
train_data["house_ownership"]=pd.factorize(train_data.house_ownership)[0]
train_data["car_ownership"]=pd.factorize(train_data.car_ownership)[0]
train_data["age"]=train_data["age"]/train_data["age"].max()
train_data["income"]=train_data["income"]/train_data["income"].max()
train_data["current_job_years"]=train_data["current_job_years"]/train_data["current_job_years"].max()
train_data["current_house_years"]=train_data["current_house_years"]/train_data["current_house_years"].max()
train_data["experience"]=train_data["experience"]/train_data["experience"].max()

In [9]:
train_data.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,0.130384,0.291139,0.15,0,0,0,0,0,0,0.214286,0.928571,0
1,2,0.757456,0.506329,0.50,0,0,0,1,1,1,0.642857,0.928571,0
2,3,0.399184,0.835443,0.20,1,0,0,2,2,2,0.285714,0.714286,0
3,4,0.625649,0.518987,0.10,0,0,1,1,3,3,0.142857,0.857143,1
4,5,0.576891,0.594937,0.55,0,0,0,3,4,4,0.214286,1.000000,1


In [10]:

train_data["risk_flag"].value_counts()

0    221004
1     30996
Name: risk_flag, dtype: int64

In [11]:
data1=train_data[train_data.risk_flag==0]
data2=train_data[train_data.risk_flag==1]

In [12]:
data1.shape

(221004, 13)

In [13]:
data2.shape

(30996, 13)

In [14]:
data3=data1.sample(n=40000)

In [15]:
data3.shape

(40000, 13)

In [16]:
trail_train_data=data3.append(data2)
trail_train_data.shape

(70996, 13)

In [17]:
trail_train_data.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
241709,241710,0.115450,0.544304,0.85,0,0,0,26,176,22,0.357143,0.857143,0
97450,97451,0.619777,0.898734,0.15,0,0,0,19,197,7,0.214286,0.928571,0
59750,59751,0.894449,0.645570,0.10,0,0,0,0,137,13,0.142857,0.785714,0
71712,71713,0.027970,0.620253,0.85,0,0,1,22,25,4,0.857143,0.928571,0
78495,78496,0.464610,0.974684,0.80,0,0,0,1,291,21,0.428571,0.928571,0


In [18]:
test_data["profession"]=pd.factorize(test_data.profession)[0]
test_data["city"]=pd.factorize(test_data.city)[0]
test_data["state"]=pd.factorize(test_data.state)[0]
test_data["married"]=pd.factorize(test_data.married)[0]
test_data["house_ownership"]=pd.factorize(test_data.house_ownership)[0]
test_data["car_ownership"]=pd.factorize(test_data.car_ownership)[0]
test_data["age"]=test_data["age"]/test_data["age"].max()
test_data["income"]=test_data["income"]/test_data["income"].max()
test_data["current_job_years"]=test_data["current_job_years"]/test_data["current_job_years"].max()
test_data["current_house_years"]=test_data["current_house_years"]/test_data["current_house_years"].max()
test_data["experience"]=test_data["experience"]/test_data["experience"].max()

In [19]:
test_data.head()

,id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
0,1,0.739323,0.746835,0.95,0,0,0,0,0,0,0.285714,0.928571
1,2,0.121503,0.316456,0.25,0,0,0,1,1,1,0.357143,0.714286
2,3,0.890151,0.632911,0.60,0,0,0,2,2,1,0.642857,1.000000
3,4,0.194446,0.620253,0.45,1,0,1,3,3,1,0.214286,0.857143
4,5,0.001343,0.316456,0.90,0,0,1,4,4,0,0.928571,0.785714


In [20]:
x_features_train=trail_train_data.drop(["risk_flag"],axis=1)
y_label_train=trail_train_data['risk_flag']

In [21]:
x_train, x_val, y_train, y_val = train_test_split(x_features_train, y_label_train, test_size=0.1, random_state=25)

In [22]:
logreg_clf=LogisticRegression()
DTree_clf=DecisionTreeClassifier(criterion='entropy', random_state=20)
SVC_clf=SVC()

In [23]:
logreg_clf.fit(x_features_train, y_label_train)
LogReg_pred = logreg_clf.predict(x_val)
log_acc=accuracy_score(y_val,LogReg_pred)
print(log_acc)

0.5629577464788732


In [24]:
log_clf_on_test=logreg_clf.predict(test_data)

In [25]:
log_clf_on_test=pd.DataFrame(log_clf_on_test)
log_clf_on_test.value_counts()

0    27987
1       13
dtype: int64

In [26]:
DTree_clf.fit(x_features_train, y_label_train)
DTree_pred = DTree_clf.predict(x_val)
DTree_acc=accuracy_score(y_val,DTree_pred)
print(DTree_acc)

1.0


In [27]:
DTree_on_test=DTree_clf.predict(test_data)

In [28]:
DTree_on_test=pd.DataFrame(DTree_on_test)
DTree_on_test.value_counts()

0    22300
1     5700
dtype: int64

In [29]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


In [30]:
clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter=100))
clf.fit(x_features_train, y_label_train)#Pipeline(steps=[('standardscaler', StandardScaler()),('sgdclassifier', SGDClassifier())])
clf_pred=clf.predict(x_val)
clf_acc=accuracy_score(y_val,clf_pred)
print(clf_acc)

0.563943661971831


In [31]:
clf_on_test=clf.predict(test_data)

In [32]:
clf_on_test=pd.DataFrame(clf_on_test)
clf_on_test.value_counts()

0    28000
dtype: int64

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
gbc_clf = GradientBoostingClassifier(n_estimators = 100)
gbc_clf.fit(x_features_train, y_label_train)
gbc_clf_pred=gbc_clf.predict(x_val)
gbc_clf_acc=accuracy_score(y_val,gbc_clf_pred)
print(gbc_clf_acc)


0.6549295774647887


In [34]:
gbc_on_test=gbc_clf.predict(test_data)
gbc_on_test=pd.DataFrame(gbc_on_test)
gbc_on_test.value_counts()

0    20424
1     7576
dtype: int64

In [35]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier(n_estimators = 90)
rfc_clf.fit(x_features_train, y_label_train)
rfc_clf_pred=rfc_clf.predict(x_val)
rfc_clf_acc=accuracy_score(y_val,rfc_clf_pred)
print(rfc_clf_acc)

1.0


In [36]:
rfc_on_test=rfc_clf.predict(test_data)
rfc_on_test=pd.DataFrame(rfc_on_test)
rfc_on_test.value_counts()

0    26989
1     1011
dtype: int64

In [37]:
from sklearn.ensemble import AdaBoostClassifier
abc_clf=AdaBoostClassifier(n_estimators = 90)
abc_clf.fit(x_features_train, y_label_train)
abc_clf_pred=abc_clf.predict(x_val)
abc_clf_acc=accuracy_score(y_val,abc_clf_pred)
print(abc_clf_acc)

0.6185915492957746


In [45]:
abc_on_test=abc_clf.predict(test_data)
abc_on_test=pd.DataFrame(abc_on_test)
abc_on_test.value_counts()

0    20900
1     7100
dtype: int64

In [47]:
from sklearn.ensemble import ExtraTreesClassifier
etc_clf=ExtraTreesClassifier(n_estimators = 90)
etc_clf.fit(x_features_train, y_label_train)
etc_clf_pred=abc_clf.predict(x_val)
etc_clf_acc=accuracy_score(y_val,etc_clf_pred)
print(etc_clf_acc)


0.6185915492957746


In [48]:
etc_on_test=etc_clf.predict(test_data)
etc_on_test=pd.DataFrame(etc_on_test)
etc_on_test.value_counts()

0    27160
1      840
dtype: int64

In [49]:
voting_clf = VotingClassifier(estimators=[('DTree_clf',DTree_clf),('abc',abc_clf ),('gbc', gbc_clf)], voting='hard')
voting_clf.fit(x_features_train, y_label_train)
preds = voting_clf.predict(x_val)
acc = accuracy_score(y_val, preds)
l_loss = log_loss(y_val, preds)
f1 = f1_score(y_val, preds)

print("Accuracy is: " + str(acc))
print("Log Loss is: " + str(l_loss))
print("F1 Score is: " + str(f1))

Accuracy is: 0.6988732394366197
Log Loss is: 10.400641296539769
F1 Score is: 0.623458964424093


In [50]:
test_preds = voting_clf.predict(test_data)

In [51]:
test_preds=pd.DataFrame(test_preds)

In [52]:
test_preds.value_counts()

0    21837
1     6163
dtype: int64

In [53]:
test_preds.head()

,0
0,1
1,1
2,1
3,0
4,1


In [55]:
test_preds["id"]=np.arange(1,28001)

In [56]:
test_preds.head()

,0,id
0,1,1
1,1,2
2,1,3
3,0,4
4,1,5


In [58]:
test_preds.rename(columns = {0:'risk_flag'}, inplace = True)

In [59]:
test_preds.head()

,risk_flag,id
0,1,1
1,1,2
2,1,3
3,0,4
4,1,5


In [60]:
test_preds=test_preds.reindex(columns= ['id','risk_flag'])

In [61]:
test_preds.head()

,id,risk_flag
0,1,1
1,2,1
2,3,1
3,4,0
4,5,1


In [62]:
test_preds.to_csv("sub2.csv")